In [1]:
############ START BOILERPLATE ############
#### Interactivity -- BOKEH
import bokeh.plotting.figure as bk_figure
from bokeh.io import curdoc, show
from bokeh.layouts import row, widgetbox, column
from bokeh.models import ColumnDataSource, LabelSet, Label
from bokeh.models.widgets import Slider, TextInput, RangeSlider
from bokeh.io import output_notebook
import numpy as np
# init bokeh

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler


output_notebook()
############ END BOILERPLATE ############

Loading BokehJS ...

In [2]:
import sys
sys.path.append("../source")
from Match_Analytics import Match
from Tracking_Visualization import draw_pitch
from Tracking_Filters import time_window
from Tracking_Statistics import bivariate_normal_distribution

m = Match(data_source = 'metrica-sports', match_id = 1)

Reading team: home
Reading team: away


In [3]:
m.preprocess()
m.calculate_player_normals()
from bokeh.models import FreehandDrawTool, TapTool
from bokeh.plotting import figure
from bokeh.layouts import gridplot
import pandas as pd

In [38]:
# Set up plot
plot = draw_pitch(tools = ('tap','pan'), toolbar_location = 'left', 
                  grass_alpha=0, background_fill_color="white", line_color="black")

renderer = plot.multi_line([], [], line_width=3, alpha=0.4, color='black')
draw_tool = FreehandDrawTool(renderers=[renderer])
plot.add_tools(draw_tool)
plot.toolbar.active_drag = draw_tool

ph = figure(toolbar_location=None, width=plot.width, height=150, x_range=plot.x_range, min_border=0, min_border_left=50, y_axis_location="left", x_axis_location = 'above')
ph.yaxis.major_label_orientation = np.pi/4
ph.background_fill_color = "white"

pv = figure(toolbar_location=None, width=150, height=plot.height,
            y_range=plot.y_range, min_border=0, y_axis_location="right", x_axis_location = 'below')
pv.xaxis.major_label_orientation = np.pi/4
pv.background_fill_color = "white"

# Set up widgets
home_time_window = RangeSlider(start=0, end=max(m.tracking_home['Time [s]']), 
                               value=(1,20), step=.4,
                               format = '00:00:00', show_value = False,
                               bar_color = 'red', height = 10)
away_time_window = RangeSlider(start=0, end=max(m.tracking_home['Time [s]']),
                               value=(1,20), step=.4,
                               format = '00:00:00', show_value = False,
                               bar_color = 'blue', height = 10)
ball_time_window = RangeSlider(start=0, end=max(m.tracking_home['Time [s]']),
                               value=(1,20), step=.4,
                               format = '00:00:00', show_value = False,
                               bar_color = 'black', height = 10)
# Set up data
hhist, hedges = np.histogram(m.tracking_home['Home_3_x'], range = (-m.field_dimen[0]/2,m.field_dimen[0]/2) , bins=int(m.field_dimen[0]), density = True)
vhist, vedges = np.histogram(m.tracking_home['Home_3_y'], range = (-m.field_dimen[1]/2,m.field_dimen[1]/2) , bins=int(m.field_dimen[1]), density = True)

ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hhist, color="red", line_color="#3A5785")
pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vhist, color="red", line_color="#3A5785")

start_home_df = time_window(m.tracking_home, 
                           home_time_window.value[0], 
                           home_time_window.value[1])
ball_dist_stats = bivariate_normal_distribution(start_home_df, ball = True)
dist_stats = bivariate_normal_distribution(start_home_df, m.home_players)
sources_home = ColumnDataSource(data=dist_stats.to_dict('series'))
sources_home_inst = ColumnDataSource(data=dict(x=[],y=[]))
sources_ball = ColumnDataSource(data=ball_dist_stats.to_dict('series'))
sources_ball_inst = ColumnDataSource(data=dict(x=[],y=[]))
    
dist_stats = bivariate_normal_distribution(time_window(m.tracking_away, 
                                                       away_time_window.value[0], 
                                                       away_time_window.value[1]), 
                                           m.away_players)
sources_away = ColumnDataSource(data=dist_stats.to_dict('series'))
sources_away_inst = ColumnDataSource(data=dict(x=[],y=[]))

plot.circle('x_mean', 'y_mean', source=sources_away, alpha=0.5, size = 15, color = "blue")
plot.circle('x_mean', 'y_mean', source=sources_home, alpha=0.5, size = 15, color = "red")
plot.circle('x_mean', 'y_mean', source=sources_ball, alpha=0.5, size = 15, color = "black")
plot.multi_line('x','y', source=sources_home_inst, color = 'red')
plot.multi_line('x','y', source=sources_away_inst, color = 'blue')
plot.line('x','y', source=sources_ball_inst, color = 'black')

labels_home = LabelSet(x='x_mean', y='y_mean', text='player',
                       source=sources_home, text_color = "white",
                       text_font_size = "10px",
                       text_baseline="middle", text_align="center")
labels_away = LabelSet(x='x_mean', y='y_mean', text='player',
                       source=sources_away, text_color = "white",
                       text_font_size = "10px", text_baseline="middle", text_align="center")
plot.ellipse(x='x_mean', 
          y='y_mean', 
          width='cov_x_std', 
          height='cov_y_std',
          source = sources_away,
          fill_color = None,
          line_width = 1,
          line_color = 'blue',
          angle = 'cov_angle')
plot.ellipse(x='x_mean', 
          y='y_mean', 
          width='cov_x_std', 
          height='cov_y_std',
          source = sources_home,
          fill_color = None,
          line_width = 1,
          line_color = "red",
          angle = 'cov_angle')
plot.ellipse(x='x_mean', 
          y='y_mean', 
          width='cov_x_std', 
          height='cov_y_std',
          source = sources_ball,
          fill_color = None,
          line_width = 1,
          line_color = "black",
          angle = 'cov_angle')
plot.add_layout(labels_home)
plot.add_layout(labels_away)

def update_data_home(attrname, old, new):
    # Get the current slider values
    c = home_time_window
    home_df = time_window(m.tracking_home, c.value[0], c.value[1])
    sources_home.data = bivariate_normal_distribution(home_df, m.home_players).to_dict('series')
    if len(sources_home.selected.indices) >= 1:
        x = []
        y = []
        for indice in sources_home.selected.indices:
            x.append(home_df[f"Home_{int(sources_home.data['player'][indice])}_x"])
            y.append(home_df[f"Home_{int(sources_home.data['player'][indice])}_y"])
        sources_home_inst.data = dict(x=x, y=y)
    else:
        sources_home_inst.data = dict(x=[],y=[])

def update_data_away(attrname, old, new):
    # Get the current slider values
    c = away_time_window
    away_df = time_window(m.tracking_away, c.value[0], c.value[1])
    sources_away.data = bivariate_normal_distribution(away_df, m.away_players).to_dict('series')
    if len(sources_away.selected.indices) >= 1:
        x = []
        y = []
        for indice in sources_away.selected.indices:
            x.append(away_df[f"Away_{int(sources_away.data['player'][indice])}_x"])
            y.append(away_df[f"Away_{int(sources_away.data['player'][indice])}_y"])
        sources_away_inst.data = dict(x=x, y=y)
    else:
        sources_away_inst.data = dict(x=[],y=[])

def update_data_ball(attrname, old, new):
    # Get the current slider values
    c = ball_time_window
    home_time_window.value = c.value
    away_time_window.value = c.value
    home_df = time_window(m.tracking_home, c.value[0], c.value[1])
    sources_ball.data = bivariate_normal_distribution(home_df, ball = True).to_dict('series')
    if len(sources_ball.selected.indices) == 1:
        sources_ball_inst.data = dict(x=home_df["ball_x"],
                                  y=home_df["ball_y"])
    else:
        sources_ball_inst.data = dict(x=[],y=[])
    
        
home_time_window.on_change('value', update_data_home)
away_time_window.on_change('value', update_data_away)
ball_time_window.on_change('value', update_data_ball)
sources_home.selected.on_change('indices', update_data_home)
sources_away.selected.on_change('indices', update_data_away)
sources_ball.selected.on_change('indices', update_data_ball)


# Set up layouts and add to document
inputs = gridplot([[ph, None],
                   [plot, pv],
                   [ball_time_window, None],
                   [home_time_window, None], 
                   [away_time_window, None]], merge_tools=False)

def modify_doc(doc):
    doc.add_root(row(inputs, width=800))


handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

In [37]:
vhist.sum()


1.0

In [6]:
pd.Series(dtype = float)

Series([], dtype: float64)

In [7]:
bivariate_normal_distribution(m.tracking_home, ball = True)

,x_mean,y_mean,normx_mean,normy_mean,cov_x_std,cov_y_std,cov_angle,cov_normx_std,cov_normy_std,cov_norm_angle,x_std,y_std,normx_std,normy_std
0,-2.177973,-1.720449,0.412645,-0.045023,25.930378,18.880427,-0.000066,1.039993,1.124906,0.182409,25.930378,18.880428,1.042897,1.122215
